In [ ]:
!pip install earthengine-api geemap pandas numpy scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.0 MB/s eta 0:00:00


In [ ]:
import geopandas as gpd
import ee
import geemap
from shapely.geometry import Polygon, MultiPolygon
import numpy as np
import pandas as pd
from shapely import wkt
from datetime import datetime
from sklearn.ensemble import IsolationForest

In [ ]:
# Step 2: Initialize Google Earth Engine
ee.Authenticate()
ee.Initialize(project='terraheal-461612')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Step 1: Simplify and preprocess the GeoJSON file
def count_vertices(geom):
    if isinstance(geom, Polygon):
        return len(geom.exterior.coords)
    elif isinstance(geom, MultiPolygon):
        return sum(len(p.exterior.coords) for p in geom.geoms)
    return 0

# Load and simplify fire boundaries
gdf = gpd.read_file("/content/drive/MyDrive/Datavista/fire.geojson").to_crs("EPSG:4326")
gdf["vertex_count"] = gdf.geometry.apply(count_vertices)
print("Max vertices:", gdf["vertex_count"].max())

# Drop overly complex polygons and simplify
gdf = gdf[gdf["vertex_count"] < 2000000]
gdf["geometry"] = gdf["geometry"].simplify(tolerance=0.0001, preserve_topology=True)
gdf = gdf[gdf.is_valid & ~gdf.is_empty]
gdf.to_file("fire_simplified.geojson", driver="GeoJSON")

Max vertices: 438


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


# Load CSV
df = pd.read_csv('/content/drive/MyDrive/Datavista/fires_dataframe.csv')

# Convert dates to YYYY-MM-DD
df['Ig_Date'] = df['Ig_Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%Y-%m-%d'))

# Parse WKT and create features
features = []
for _, row in df.iterrows():
    geom = wkt.loads(row['geometry'])
    geojson = geom.__geo_interface__
    ee_geom = ee.Geometry(geojson)
    feature = ee.Feature(ee_geom, {
        'name': row['Incid_Name'],  # Adjust based on code's expected property
        'Ig_Date': row['Ig_Date']
    })
    features.append(feature)

# Create FeatureCollection
fc = ee.FeatureCollection(features)

In [ ]:


# Load simplified GeoJSON as an Earth Engine feature
aoi = geemap.geojson_to_ee("fire_simplified.geojson")

# Step 3: Sentinel-2 NDVI Calculation
def mask_clouds(img):
    qa = img.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0))
    return img.updateMask(cloud_mask)

def add_ndvi_and_time(img):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
    time = img.date().millis().rename('time')
    return img.addBands([ndvi, time])

In [ ]:
# Step 3: Sentinel-2 NDVI Calculation
# Keep the existing mask_scl and add_ndvi functions

def mask_scl(img):
    # SCL classes: 3 = Cloud shadow, 8 = Medium probability cloud, 9 = High prob. cloud, 10 = Thin cirrus, 11 = Snow
    scl = img.select('SCL')
    mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)).And(scl.neq(11))
    return img.updateMask(mask)

def add_ndvi_and_time(img):
    img = mask_clouds(img)
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
    time = ee.Image(img.date().millis()).toFloat().rename('time')  # Cast to consistent type
    return img.addBands([ndvi, time])


# Load Sentinel-2 imagery for pre-fire and post-fire periods
pre_fire = (
    ee.ImageCollection("COPERNICUS/S2_SR")
    .filterBounds(aoi)
    .filterDate("2018-08-01", "2020-07-31")
    .map(mask_clouds)  # Cloud masking
    .map(add_ndvi_and_time)  # NDVI + time band
)
pre_ndvi = pre_fire.select("NDVI").median().clip(aoi)
# Apply the same processing as pre-fire
post_fire = (
    ee.ImageCollection("COPERNICUS/S2_SR")
    .filterBounds(aoi)
    .filterDate("2020-08-01", "2022-01-01")  # Fire + recovery period
    .map(mask_clouds)
    .map(add_ndvi_and_time)
)

# Reduce to median NDVI and clip
post_ndvi = post_fire.select("NDVI").median().clip(aoi)

# Calculate NDVI difference (post - pre)
ndvi_diff = post_ndvi.subtract(pre_ndvi).rename('NDVI_diff')

In [ ]:
# Step 4: Anomaly Detection for Cold Spots
# Compute mean and standard deviation of NDVI difference within the AOI
# Perform linear regression to get recovery slope
linear_fit = post_fire.select(['time', 'NDVI']).reduce(ee.Reducer.linearFit())
slope = linear_fit.select('scale').clip(aoi)

# Anomaly detection: Calculate mean and std dev of slopes, then z-scores
slope_stats = slope.reduceRegion(
    reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True),
    geometry=aoi,
    scale=20,
    maxPixels=1e9
)
mean_slope = ee.Number(slope_stats.get('scale_mean'))
std_slope = ee.Number(slope_stats.get('scale_stdDev'))
z_score = slope.subtract(mean_slope).divide(std_slope)

# Define cold spots as areas with z-score < -2 (significant negative deviation)
cold_spots = z_score.lt(-2).selfMask()

# Mask to areas with pre-fire NDVI > 0.3 (previously vegetated)
vegetated_mask = pre_ndvi.gt(0.3)
cold_spots = cold_spots.updateMask(vegetated_mask)

In [ ]:
# Visualize the results
Map = geemap.Map()
Map.centerObject(aoi, 9)
Map.addLayer(ndvi_diff, {"min": -0.5, "max": 0.5, "palette": ["red", "white", "green"]}, "NDVI Change")
Map.addLayer(slope, {'min': -0.01, 'max': 0.01, 'palette': ['red', 'white', 'green']}, 'Recovery Slope')
Map.addLayer(cold_spots, {'palette': 'blue'}, 'Cold Spots')
Map.addLayer(aoi, {}, "Fire Boundary")
Map

Map(center=[43.120908260522825, -116.78567588786129], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
# Step 6: Export Cold Spots (Optional)
export_task = ee.batch.Export.image.toDrive(
    image=cold_spots,
    description='Cold_Spots_Recovery',
    folder='Datavista',
    region=aoi.geometry(),
    scale=20,
    crs='EPSG:4326',
    maxPixels=1e9
)
export_task.start()
print("Exporting cold spots to Google Drive...")

Exporting cold spots to Google Drive...


In [ ]:
import geopandas as gpd
import ee
import geemap
from shapely.geometry import Polygon, MultiPolygon
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

# Step 1: Simplify and preprocess the GeoJSON file
def count_vertices(geom):
    if isinstance(geom, Polygon):
        return len(geom.exterior.coords)
    elif isinstance(geom, MultiPolygon):
        return sum(len(p.exterior.coords) for p in geom.geoms)
    return 0

# Load and simplify fire boundaries
gdf = gpd.read_file("/content/drive/MyDrive/Datavista/fire.geojson").to_crs("EPSG:4326")
gdf["vertex_count"] = gdf.geometry.apply(count_vertices)
print("Max vertices:", gdf["vertex_count"].max())

# Drop overly complex polygons and simplify
gdf = gdf[gdf["vertex_count"] < 2000000]
gdf["geometry"] = gdf["geometry"].simplify(tolerance=0.0001, preserve_topology=True)
gdf = gdf[gdf.is_valid & ~gdf.is_empty]
gdf.to_file("fire_simplified.geojson", driver="GeoJSON")

# Step 2: Initialize Google Earth Engine
ee.Authenticate()
ee.Initialize(project='terraheal-461612')

# Load simplified GeoJSON as an Earth Engine feature
aoi = geemap.geojson_to_ee("fire_simplified.geojson")

# Step 3: NDVI Preprocessing and Feature Engineering
def mask_clouds(img):
    qa = img.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0))
    return img.updateMask(cloud_mask)

def add_ndvi(img):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return img.addBands(ndvi)

# Calculate baseline NDVI (2018–2019)
pre_fire = (
    ee.ImageCollection("COPERNICUS/S2_SR")
    .filterBounds(aoi)
    .filterDate("2018-01-01", "2019-12-31")
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .map(mask_clouds)
    .map(add_ndvi)
    .select("NDVI")
    .median()
    .rename('baseline_ndvi')
)

# Calculate post-fire NDVI at multiple intervals (t0, t3, t6, t12, t18, t24)
time_points = [
    ("2020-08-01", "2020-10-31", "ndvi_t0"),   # Immediately after fire
    ("2020-11-01", "2021-01-31", "ndvi_t3"),   # 3 months
    ("2021-02-01", "2021-04-30", "ndvi_t6"),   # 6 months
    ("2021-07-01", "2021-09-30", "ndvi_t12"),  # 12 months
    ("2022-01-01", "2022-03-31", "ndvi_t18"),  # 18 months
    ("2022-07-01", "2022-09-30", "ndvi_t24")   # 24 months
]

ndvi_images = []
for start_date, end_date, name in time_points:
    img = (
        ee.ImageCollection("COPERNICUS/S2_SR")
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
        .map(mask_clouds)
        .map(add_ndvi)
        .select("NDVI")
        .median()
        .rename(name)
    )
    ndvi_images.append(img)

# Combine all NDVI images into a single multi-band image
ndvi_stack = ee.Image.cat([pre_fire] + ndvi_images)

# Calculate NDVI drop and recovery rate
ndvi_drops = [ndvi_stack.select('baseline_ndvi').subtract(ndvi_stack.select(name)).rename(f'ndvi_drop_{name}') for name in [t[2] for t in time_points]]
recovery_rate_t3_t12 = ndvi_stack.select('ndvi_t12').subtract(ndvi_stack.select('ndvi_t3')).divide(9).rename('recovery_rate_t3_t12')
recovery_rate_t12_t24 = ndvi_stack.select('ndvi_t24').subtract(ndvi_stack.select('ndvi_t12')).divide(12).rename('recovery_rate_t12_t24')

# Combine all features
feature_stack = ee.Image.cat([ndvi_stack] + ndvi_drops + [recovery_rate_t3_t12, recovery_rate_t12_t24])

# Step 4: Heuristic Cold Spot Labeling
# Rule 1: NDVI at t12 < 50% of baseline
threshold = pre_fire.multiply(0.5)
cold_spots_threshold = ndvi_stack.select('ndvi_t12').lt(threshold).rename('cold_spot_threshold')

# Calculate the 25th percentile of recovery_rate_t3_t12
stats = recovery_rate_t3_t12.reduceRegion(
    reducer=ee.Reducer.percentile([25]),
    geometry=aoi,
    scale=20,
    maxPixels=1e9,
    bestEffort=True  # Automatically adjust scale if needed
)

# Debugging: Print the stats dictionary to inspect keys
print("Stats dictionary:", stats.getInfo())

# Extract the 25th percentile (adjust key based on actual output)
# GEE might name the key as just 'p25' if the band name is not prepended
percentile_25 = ee.Number(stats.get('p25', stats.get('recovery_rate_t3_t12_p25', 0)))
cold_spots_percentile = recovery_rate_t3_t12.lt(percentile_25).rename('cold_spot_percentile')

# Combine rules (cold spot if either condition is met)
cold_spots_heuristic = cold_spots_threshold.Or(cold_spots_percentile).rename('cold_spot_heuristic')

# Step 5: Anomaly Detection Model (Isolation Forest)
# Export features for modeling
feature_names = ['baseline_ndvi', 'ndvi_t0', 'ndvi_t3', 'ndvi_t6', 'ndvi_t12', 'ndvi_t18', 'ndvi_t24',
                 'ndvi_drop_ndvi_t0', 'ndvi_drop_ndvi_t3', 'ndvi_drop_ndvi_t6', 'ndvi_drop_ndvi_t12',
                 'ndvi_drop_ndvi_t18', 'ndvi_drop_ndvi_t24', 'recovery_rate_t3_t12', 'recovery_rate_t12_t24']

export_task = ee.batch.Export.image.toDrive(
    image=feature_stack,
    description='NDVI_Features',
    folder='GEE_Exports',
    region=aoi.geometry(),
    scale=20,
    crs='EPSG:4326',
    maxPixels=1e10
)
export_task.start()
print("Exporting features to Google Drive... Please wait and download NDVI_Features.csv")

Max vertices: 438
Stats dictionary: {'recovery_rate_t3_t12': 0.0017170270283848764}
Exporting features to Google Drive... Please wait and download NDVI_Features.csv


In [ ]:

# Assuming you download the CSV, load it in Colab
# Replace 'NDVI_Features.csv' with the path to your downloaded file
data = pd.read_csv('/content/drive/MyDrive/GEE_Exports/NDVI_Features.tif')
features = data[feature_names]
iso_forest = IsolationForest(contamination=0.1, random_state=42)
cold_spot_labels = iso_forest.fit_predict(features)
data['cold_spot_iso_forest'] = cold_spot_labels == -1

# Step 6: Verify Cold Spots with Extended Timestamp
# Check if cold spots at t12 persist at t24
cold_spots_t24 = ndvi_stack.select('ndvi_t24').lt(threshold).rename('cold_spot_t24')
persistent_cold_spots = cold_spots_heuristic.And(cold_spots_t24).rename('persistent_cold_spot')

# Step 7: Visualize Results
Map = geemap.Map()
Map.centerObject(aoi, 9)
Map.addLayer(pre_fire, {"min": 0, "max": 1, "palette": ["brown", "yellow", "green"]}, "Baseline NDVI")
Map.addLayer(ndvi_stack.select('ndvi_t12'), {"min": 0, "max": 1, "palette": ["brown", "yellow", "green"]}, "NDVI t12")
Map.addLayer(cold_spots_heuristic, {"palette": ["white", "red"]}, "Cold Spots (Heuristic)")
Map.addLayer(persistent_cold_spots, {"palette": ["white", "purple"]}, "Persistent Cold Spots (t24)")
Map.addLayer(aoi, {}, "Fire Boundary")
Map

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc8 in position 36: invalid continuation byte

In [ ]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 56.4 MB/s eta 0:00:00


In [ ]:
# Assuming you download the TIFF file, load it in Colab using rasterio
# Replace 'NDVI_Features.tif' with the path to your downloaded file

# Install rasterio if not already installed


import rasterio
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

# Step 5: Anomaly Detection Model (Isolation Forest)
# Export features for modeling is done in the previous cell, producing NDVI_Features.tif

# --- Start of Suggested Change ---

# Read the TIFF file using rasterio
try:
    with rasterio.open('/content/drive/MyDrive/GEE_Exports/NDVI_Features.tif') as src:
        # Read all bands into a numpy array
        # The shape will be (bands, height, width)
        image_data = src.read()

        # Get the nodata value if present
        nodata = src.nodata

    # Reshape the image data for pandas: (height * width, bands)
    # Transpose (bands, height, width) to (height, width, bands) first
    image_data_reshaped = np.moveaxis(image_data, 0, -1)
    # Flatten the spatial dimensions to get pixel values and reshape to (n_pixels, n_bands)
    pixels = image_data_reshaped.reshape(-1, image_data_reshaped.shape[-1])

    # Create a DataFrame from the pixel data
    # Ensure the number of columns matches the number of bands read
    num_bands = image_data.shape[0]
    # Use generic names if feature_names list length doesn't match num_bands
    if len(feature_names) == num_bands:
        column_names = feature_names
    else:
        # Fallback to generic column names if feature_names mismatch
        column_names = [f'band_{i+1}' for i in range(num_bands)]
        print(f"Warning: Number of bands ({num_bands}) in TIFF does not match "
              f"length of feature_names ({len(feature_names)}). Using generic band names.")

    data = pd.DataFrame(pixels, columns=column_names)

    # Handle NoData values: replace with NaN
    if nodata is not None:
        data = data.replace(nodata, np.nan)

    # Drop rows with NaN values (pixels outside the AOI or masked)
    data = data.dropna()

    # Ensure required features are present
    try:
        features = data[feature_names]
    except KeyError as e:
        print(f"Error: Missing expected feature column: {e}")
        print(f"Available columns: {data.columns.tolist()}")
        # Decide how to handle this: e.g., exit, use available columns, etc.
        # For now, let's stop execution
        raise

except FileNotFoundError:
    print("Error: NDVI_Features.tif not found at the specified path.")
    # Decide how to handle this: e.g., exit, wait for export, etc.
    # For now, let's stop execution
    raise
except Exception as e:
    print(f"An error occurred while reading or processing the TIFF file: {e}")
    # Decide how to handle other exceptions
    # For now, re-raise the exception
    raise


# Now proceed with the Isolation Forest model using the DataFrame 'features'
iso_forest = IsolationForest(contamination=0.1, random_state=42)
cold_spot_labels = iso_forest.fit_predict(features)

# Map the Isolation Forest results back to the original pixel locations if needed for visualization
# This requires more complex handling if dropped NaNs or flattened data
# For simplicity here, we'll just add the labels to the DataFrame used for training
# This assumes the indices of 'data' correspond to the valid pixels from the TIFF
data['cold_spot_iso_forest'] = cold_spot_labels == -1

# --- End of Suggested Change ---


# Step 6: Verify Cold Spots with Extended Timestamp
# Note: This step currently relies on GEE objects (ndvi_stack, threshold)
# and cannot directly use the local pandas DataFrame 'data' and 'cold_spot_iso_forest'.
# To fully integrate local model results, you would need to:
# 1. Create an Earth Engine Image from the 'cold_spot_iso_forest' column
#    with the correct spatial reference and extent matching the original TIFF.
# 2. Use this EE Image of model results in combination with the EE Images (ndvi_stack, threshold)
#    for further analysis within Earth Engine.
#
# For now, let's assume this step needs to be re-evaluated or adapted.
# The original code uses GEE objects, which is appropriate if the goal is to perform
# this analysis within Earth Engine or integrate with GEE-based visualizations.
# The local Isolation Forest model provides an alternative approach but linking
# its output back to GEE for spatial operations requires more steps.
#
# Keeping the original GEE-based Step 6 for reference, but note it doesn't
# directly use the results from the local Isolation Forest model run above.

# Check if cold spots at t12 persist at t24 (This is using GEE objects)
cold_spots_t24 = ndvi_stack.select('ndvi_t24').lt(threshold).rename('cold_spot_t24')
persistent_cold_spots = cold_spots_heuristic.And(cold_spots_t24).rename('persistent_cold_spot')

# Step 7: Visualize Results
# This step uses GEE objects for visualization.
# To visualize the results of the local Isolation Forest model, you would need
# to either export the result DataFrame with coordinates and visualize in a local
# GIS tool, or create an EE Image from the model results and add it to the map.

Map = geemap.Map()
Map.centerObject(aoi, 9)
Map.addLayer(pre_fire, {"min": 0, "max": 1, "palette": ["brown", "yellow", "green"]}, "Baseline NDVI")
Map.addLayer(ndvi_stack.select('ndvi_t12'), {"min": 0, "max": 1, "palette": ["brown", "yellow", "green"]}, "NDVI t12")

# Add the heuristic cold spots layer (from GEE object)
Map.addLayer(cold_spots_heuristic, {"palette": ["white", "red"]}, "Cold Spots (Heuristic)")

# If you created an EE Image from the local Isolation Forest results, add it here
# Example (conceptual, requires implementing the EE Image creation):
#iso_forest_ee_image = create_ee_image_from_dataframe(data, 'cold_spot_iso_forest', original_tiff_bounds, original_tiff_crs)
#Map.addLayer(iso_forest_ee_image.mask(iso_forest_ee_image.gt(0)), {"palette": "blue"}, "Cold Spots (Isolation Forest)")

# Add the persistent cold spots layer (from GEE object)
Map.addLayer(persistent_cold_spots, {"palette": ["white", "purple"]}, "Persistent Cold Spots (t24)")

Map.addLayer(aoi, {}, "Fire Boundary")
Map

Map(center=[43.120908260522825, -116.78567588786129], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
!pip install ipywidgets geemap pandas shapely earthengine-api -q
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import pandas as pd
from shapely import wkt
from datetime import datetime, timedelta

# --- Try to Initialize GEE ---
try:
    ee.Initialize() # Colab often handles auth smoothly
    print("GEE Initialized")
except Exception as e:
    try:
        ee.Authenticate()
        ee.Initialize(project='terraheal-461612') # Replace if needed
        print("GEE Initialized after auth")
    except Exception as auth_e:
        print(f"GEE Initialization failed: {auth_e}. Please authenticate manually if prompted.")
        raise

# --- Mockup data and GEE functions (simplified from your Flask app) ---
# Assume fires_dataframe.csv is uploaded to Colab or accessible via Drive
# For this example, let's create a dummy DataFrame
data = {
    'Incid_Name': ['CAMP FIRE', 'DIXIE FIRE', 'AUGUST COMPLEX'],
    'Ig_Date': ['2018-11-08', '2021-07-13', '2020-08-17'],
    'geometry': [ # Simplified WKTs for example, use your actual complex ones
        'POLYGON((-121.5 39.8, -121.4 39.8, -121.4 39.9, -121.5 39.9, -121.5 39.8))',
        'POLYGON((-121.0 40.0, -120.9 40.0, -120.9 40.1, -121.0 40.1, -121.0 40.0))',
        'POLYGON((-122.8 39.7, -122.7 39.7, -122.7 39.8, -122.8 39.8, -122.8 39.7))'
    ]
}
fires_df = pd.DataFrame(data)

def get_fire_data_colab(fire_name_csv):
    fire_row = fires_df[fires_df['Incid_Name'] == fire_name_csv]
    if fire_row.empty:
        raise ValueError(f"Fire '{fire_name_csv}' not found.")
    geometry_wkt = fire_row.iloc[0]['geometry']
    ignition_date_str = fire_row.iloc[0]['Ig_Date']
    shapely_geom = wkt.loads(geometry_wkt)
    geojson_geom = shapely_geom.__geo_interface__
    aoi = ee.Geometry(geojson_geom).simplify(maxError=100)
    return aoi, ignition_date_str

def generate_map_for_fire_colab(aoi, ignition_date_str):
    # Simplified map generation - use your more complex logic
    Map = geemap.Map(center=aoi.centroid().coordinates().getInfo()[::-1], zoom=9) # Reverse for lat,lon
    Map.addLayer(aoi, {}, "Fire Boundary")

    # Example: Add an NDVI layer (very basic)
    try:
        ig_date = datetime.strptime(ignition_date_str, '%Y-%m-%d')
        start_date = (ig_date - timedelta(days=30)).strftime('%Y-%m-%d')
        end_date = ig_date.strftime('%Y-%m-%d')

        s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterBounds(aoi) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .first() # Get a single image for simplicity

        if s2.getInfo(): # Check if an image was found
            ndvi = s2.normalizedDifference(['B8', 'B4']).rename('NDVI')
            Map.addLayer(ndvi.clip(aoi), {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDVI (Pre-fire example)')
        else:
            print(f"No suitable S2 image found for {ignition_date_str} period.")
    except Exception as e:
        print(f"Error adding NDVI layer: {e}")

    Map.addLayerControl()
    return Map

# --- Create Widgets ---
fire_options = fires_df['Incid_Name'].tolist()
fire_dropdown = widgets.Dropdown(
    options=fire_options,
    value=fire_options[0],
    description='Select Fire:',
    disabled=False,
)

map_output_area = widgets.Output() # Area to display the map

# --- Define the function to be called on widget change ---
def on_fire_selected(change):
    with map_output_area:
        clear_output(wait=True) # Clear previous map
        selected_fire_name = change['new']
        print(f"Loading map for: {selected_fire_name}...")
        try:
            aoi, ig_date_str = get_fire_data_colab(selected_fire_name)
            gee_map_object = generate_map_for_fire_colab(aoi, ig_date_str)
            # geemap.Map objects can be directly displayed in Colab output cells
            display(gee_map_object)
            print("Map loaded.")
        except Exception as e:
            print(f"Error generating map: {e}")

fire_dropdown.observe(on_fire_selected, names='value')

# --- Display the UI ---
display(HTML("<h2>Wildfire Recovery Dashboard (In-Notebook)</h2>"))
display(fire_dropdown)
display(map_output_area)

# --- Initial map load ---
with map_output_area: # Trigger initial load
    print(f"Loading initial map for: {fire_dropdown.value}...")
    try:
        aoi, ig_date_str = get_fire_data_colab(fire_dropdown.value)
        gee_map_object = generate_map_for_fire_colab(aoi, ig_date_str)
        display(gee_map_object)
        print("Initial map loaded.")
    except Exception as e:
        print(f"Error generating initial map: {e}")

GEE Initialized after auth


Dropdown(description='Select Fire:', options=('CAMP FIRE', 'DIXIE FIRE', 'AUGUST COMPLEX'), value='CAMP FIRE')

Output()

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import pandas as pd
from shapely import wkt
from datetime import datetime, timedelta
import ee
import geemap

# --- Try to Initialize GEE ---
try:
    ee.Initialize() # Colab often handles auth smoothly
    print("GEE Initialized")
except Exception as e:
    try:
        # You might need to manually authenticate if the above fails
        # ee.Authenticate()
        ee.Initialize(project='terraheal-461612') # Replace with your GCP project ID
        print("GEE Initialized after auth")
    except Exception as auth_e:
        print(f"GEE Initialization failed: {auth_e}. Please ensure you have authenticated and specified a project.")
        raise

# --- Load Fire Data ---
# Ensure 'fires_dataframe.csv' is accessible. You can upload it or mount Google Drive.
try:
    fires_df = pd.read_csv('/content/drive/MyDrive/Datavista/fires_dataframe.csv')
    # Ensure the 'Ig_Date' is in a consistent format (YYYY-MM-DD)
    fires_df['Ig_Date'] = pd.to_datetime(fires_df['Ig_Date']).dt.strftime('%Y-%m-%d')
    print("Fire data loaded successfully.")
except FileNotFoundError:
    print("Error: fires_dataframe.csv not found. Please ensure the file is in the specified path.")
    # Create a dummy DataFrame if the file is not found to allow the UI to load
    data = {
        'Incid_Name': ['Dummy Fire 1', 'Dummy Fire 2'],
        'Ig_Date': ['2022-01-01', '2023-06-15'],
        'geometry': [
            'POLYGON((-120 40, -119 40, -119 41, -120 41, -120 40))',
            'POLYGON((-122 38, -121 38, -121 39, -122 39, -122 38))'
        ]
    }
    fires_df = pd.DataFrame(data)
    print("Using dummy fire data.")
except Exception as e:
    print(f"An error occurred while loading fire data: {e}")
    # Create a dummy DataFrame if loading fails
    data = {
        'Incid_Name': ['Dummy Fire 1', 'Dummy Fire 2'],
        'Ig_Date': ['2022-01-01', '2023-06-15'],
        'geometry': [
            'POLYGON((-120 40, -119 40, -119 41, -120 41, -120 40))',
            'POLYGON((-122 38, -121 38, -121 39, -122 39, -122 38))'
        ]
    }
    fires_df = pd.DataFrame(data)
    print("Using dummy fire data due to error.")


def get_fire_data(fire_name):
    """Retrieves fire geometry and ignition date from the DataFrame."""
    fire_row = fires_df[fires_df['Incid_Name'] == fire_name]
    if fire_row.empty:
        raise ValueError(f"Fire '{fire_name}' not found in DataFrame.")
    geometry_wkt = fire_row.iloc[0]['geometry']
    ignition_date_str = fire_row.iloc[0]['Ig_Date']
    try:
        shapely_geom = wkt.loads(geometry_wkt)
        geojson_geom = shapely_geom.__geo_interface__
        # Simplify geometry to reduce complexity for GEE processing
        aoi = ee.Geometry(geojson_geom).simplify(maxError=100)
    except Exception as e:
        print(f"Error processing geometry for {fire_name}: {e}")
        # Return a default point or small bounding box if geometry is invalid
        aoi = ee.Geometry.Point([-120, 40]).buffer(1000) # Example: a point near California
        print(f"Using a default point geometry for {fire_name} due to error.")
    return aoi, ignition_date_str

def mask_clouds_and_shadows(img):
    """Masks clouds and cloud shadows using the SCL band."""
    # SCL classes: 3 = Cloud shadow, 8 = Medium probability cloud, 9 = High prob. cloud, 10 = Thin cirrus, 11 = Snow
    scl = img.select('SCL')
    mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)).And(scl.neq(11))
    return img.updateMask(mask)

def add_ndvi(img):
    """Calculates and adds an NDVI band to the image."""
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return img.addBands(ndvi)

def generate_fire_dashboard_map(aoi, ignition_date_str):
    """Generates a geemap Map object with relevant fire analysis layers."""
    Map = geemap.Map(center=aoi.centroid().coordinates().getInfo()[::-1], zoom=9)

    ig_date = datetime.strptime(ignition_date_str, '%Y-%m-%d')

    # --- Add Baseline NDVI (Pre-fire) ---
    pre_fire_start = (ig_date - timedelta(days=365*2)).strftime('%Y-%m-%d') # 2 years before
    pre_fire_end = (ig_date - timedelta(days=30)).strftime('%Y-%m-%d') # Up to 1 month before

    try:
        pre_fire_collection = (
            ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
            .filterBounds(aoi)
            .filterDate(pre_fire_start, pre_fire_end)
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
            .map(mask_clouds_and_shadows)
            .map(add_ndvi)
        )
        pre_fire_ndvi = pre_fire_collection.select("NDVI").median().clip(aoi)
        if pre_fire_ndvi.getInfo(): # Check if an image was produced
             Map.addLayer(pre_fire_ndvi, {"min": 0, "max": 1, "palette": ["brown", "yellow", "green"]}, "Baseline NDVI")
        else:
            print("Warning: No pre-fire imagery found for the selected period.")
            # Add a grey layer or message indicating no data
            no_data_img = ee.Image(0).mask(ee.Image(0)).paint(aoi, 1) # Invisible mask, painted boundary
            Map.addLayer(no_data_img, {'palette': 'gray'}, "Baseline NDVI (No Data)")
    except Exception as e:
        print(f"Error adding Baseline NDVI layer: {e}")


    # --- Add Post-fire NDVI Change (Example: 1 year after) ---
    post_fire_start = (ig_date + timedelta(days=30)).strftime('%Y-%m-%d') # 1 month after
    post_fire_end = (ig_date + timedelta(days=365 + 30)).strftime('%Y-%m-%d') # ~1 year after

    try:
        post_fire_collection = (
            ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
            .filterBounds(aoi)
            .filterDate(post_fire_start, post_fire_end)
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
            .map(mask_clouds_and_shadows)
            .map(add_ndvi)
        )
        post_fire_ndvi = post_fire_collection.select("NDVI").median().clip(aoi)

        if pre_fire_ndvi.getInfo() and post_fire_ndvi.getInfo(): # Check if both pre and post images exist
            ndvi_diff = post_fire_ndvi.subtract(pre_fire_ndvi).rename('NDVI_diff')
            # Define appropriate visualization parameters for NDVI difference
            # Values typically range from -1 to 1, but change due to fire is often -0.8 to 0.2
            # Palette: Red (decrease), White (no change), Green (increase)
            vis_params_diff = {
                "min": -0.5,
                "max": 0.5,
                "palette": ["FF0000", "FFFFFF", "00FF00"] # Red, White, Green
            }
            Map.addLayer(ndvi_diff, vis_params_diff, "NDVI Change (~1 Year Post-fire)")
        elif post_fire_ndvi.getInfo():
             Map.addLayer(post_fire_ndvi, {"min": 0, "max": 1, "palette": ["brown", "yellow", "green"]}, "NDVI (~1 Year Post-fire)")
        else:
            print("Warning: No post-fire imagery found for the selected period to calculate NDVI change.")

    except Exception as e:
        print(f"Error adding NDVI Change layer: {e}")


    # --- Add Fire Boundary ---
    Map.addLayer(aoi.buffer(1000), {'color': 'red', 'fillColor': '00000000'}, "Fire Boundary (Buffered)") # Buffer slightly for visibility

    # --- Add Layer Control ---
    Map.addLayerControl()

    return Map

# --- Create Widgets ---
# Get fire options from the loaded DataFrame
fire_options = fires_df['Incid_Name'].tolist()

if not fire_options:
    fire_options = ["No Fire Data Available"] # Handle case where DataFrame is empty or dummy data used

fire_dropdown = widgets.Dropdown(
    options=fire_options,
    value=fire_options[0],
    description='Select Fire:',
    disabled=False,
)

map_output_area = widgets.Output() # Area to display the map

# --- Define the function to be called on widget change ---
def on_fire_selected(change):
    with map_output_area:
        clear_output(wait=True) # Clear previous map
        selected_fire_name = change['new']
        if selected_fire_name == "No Fire Data Available":
            print("Cannot generate map: No fire data loaded.")
            return

        print(f"Loading map for: {selected_fire_name}...")
        try:
            aoi, ig_date_str = get_fire_data(selected_fire_name)
            gee_map_object = generate_fire_dashboard_map(aoi, ig_date_str)
            # geemap.Map objects can be directly displayed in Colab output cells
            display(gee_map_object)
            print("Map loaded.")
        except Exception as e:
            print(f"Error generating map: {e}")
            # Display an error message on the map area
            display(HTML(f"<p style='color:red;'>Error generating map: {e}</p>"))


# Observe changes in the dropdown value
fire_dropdown.observe(on_fire_selected, names='value')

# --- Display the UI ---
display(HTML("<h2>Wildfire Recovery Dashboard</h2>"))
display(fire_dropdown)
display(map_output_area)

# --- Initial map load ---
# Trigger the function once initially to display the map for the default selected fire
if fire_options and fire_options[0] != "No Fire Data Available":
    on_fire_selected({'new': fire_dropdown.value})
else:
    with map_output_area:
        display(HTML("<p>Please load the fire data (fires_dataframe.csv) to use the dashboard.</p>"))

GEE Initialized after auth
Fire data loaded successfully.


Dropdown(description='Select Fire:', options=('CORRAL', 'SNF GOOBER HILL S BLKB/C RX', 'UFISH', 'SNF GOOBER HI…

Output()